# Gradients

NNsight supports performing and intervening on backwards passes, which enables "get" and "set" operations on gradients.

### General usage:
```
with M.trace("hi"):

  # Grab values from forward pass (necessary)
  a = M.a.output
  b = M.b.output
  
  with f(M.b.output).backward():
      grads = b.grad ## GET
      a.grad = grads ## SET
```

(Note, for this to work, submodule $a$ must come before submodule $b$ in the forward pass)

### Example

In [10]:
from nnsight import LanguageModel
import torch

model = LanguageModel("openai-community/gpt2", device_map="auto")

with model.trace("Hello World") as tracer:

    a = model.transformer.h[-2].output[0]
    b = model.transformer.h[-1].output[0]
    
    with model.output.logits.sum().backward():
        b_grad = b.grad.save()
        a.grad = b_grad

In [11]:
print(b_grad)

tensor([[[  28.7976, -282.5977,  868.7343,  ...,  120.1742,   52.2263,
           168.6448],
         [  79.4182, -253.6227, 1322.1293,  ...,  208.3982,  -19.5545,
           509.9859]]], device='cuda:0')




In order to use multiple `.backward()` contexts in a single trace context, each subsequent `.backward()` requires the keyword argument `retain_graph=True` set.